In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

import sys
sys.path.append('../')
from utils import *

In [2]:
watches = pd.read_pickle('../../data/user-level-data')
videos = pd.read_pickle('../../data/videos_raw_metadata')

In [36]:
watches_df = dict()

def convert(watch):
    d = dict()
    d['playing'] = watch['playing']['id']
    suggested = []
    for s in watch['suggested']:
        if s != None:
            suggested.append(s['id'])
    d['suggested'] = suggested
    d['selected'] = watch['selected']
    return d

for user in watches:
    watches_df[user] = []
    for watch in watches[user]:
        if watch['playing'] == None:
            continue
        watch = convert(watch)
        watches_df[user].append(watch)

In [37]:
history = dict()
users = list(watches_df.keys())
data = dict()

for user in users:
    if len(watches_df[user]) < 10:
        continue
    history[user] = watches_df[user][:10]

    for watch in watches_df[user][10:]:
        if watch['selected'] != None:
            if user not in data:
                data[user] = []
            data[user].append(watch)

for user in list(history.keys()):
    if user not in data:
        history.pop(user)

In [38]:
for user in history:
    history[user] = [s['playing'] for s in history[user]]
    history[user] = [get_topic_vector(s) for s in history[user]]
    history[user] = np.array(history[user]).mean(axis=0)

In [35]:
from collections import Counter


all_topics = []
all_tags = []

for video in videos:
    video = videos[video]
    
    try:
        topics = video['topicDetails']['topicCategories']
        topics = [topic.split('/')[-1].lower() for topic in topics]
    except:
        topics = []

    all_topics += topics
    
    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    all_tags += tags

all_topics = list(set(all_topics))

atc = Counter(all_tags)
atc = pd.DataFrame.from_dict(atc, 'index', columns=['count'])
atc = atc.sort_values(['count'], ascending=False).head(5000)
all_tags = list(atc.index)

def get_one_hot_vector(topics, tags):
    oh_topics = np.zeros(len(all_topics))
    for topic in topics:
        oh_topics[all_topics.index(topic)] = 1
        
    oh_tags = np.zeros(len(all_tags))
    for tag in tags:
        if tag in all_tags:
            oh_tags[all_tags.index(tag)] = 1
        
    oh = np.concatenate([oh_topics , oh_tags])
    
    return oh


def get_topic_vector(video):
    video = videos[video]
    try:
        topics = video['topicDetails']['topicCategories']
        topics = [topic.split('/')[-1].lower() for topic in topics]
    except:
        topics = []

    try:
        tags = video['snippet']['tags']
        tags = [tag.lower() for tag in tags]
    except:
        tags = []

    return get_one_hot_vector(topics, tags)


In [39]:
X = []
Y = []

for user in tqdm(data):
    user_history = history[user]
    for watch in data[user]:
        playing = watch['playing']
        selected = watch['selected']
        upnext = watch['suggested'][:15]

        if playing not in videos or selected not in videos or any([s not in videos for s in watch['suggested']]):
            continue
        if len(watch['suggested']) < 15:
            continue


        playing = get_topic_vector(watch['playing'])

        suggesteds = []
        y = []
        for i, s in enumerate(upnext):
            selected = watch['selected']

            if s == selected:
                y.append(1)
            else:
                y.append(0)

            sv = get_topic_vector(s)
            sv = np.concatenate([sv, [i]])
            suggesteds.append(sv)
        
        suggesteds = np.array(suggesteds).flatten()
        x = np.concatenate([user_history, playing, suggesteds])
    
        X.append(x)
        Y.append(y)

  0%|          | 0/1266 [00:00<?, ?it/s]

In [40]:
X = np.array(X)
Y = np.array(Y)

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [29]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [30]:
pos = Y.sum()
neg = len(Y) - pos
total = neg + pos
w0 = (1 / neg) * (total / 2.0)
w1 = (1 / pos) * (total / 2.0)
class_weights = {0:w0, 1:w1}
class_weights

{0: 16.521141649048626, 1: 0.5156043811031935}

In [44]:
X_train.shape

(12503, 86069)

In [45]:
# train neural network

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD


model = Sequential()
model.add(Dense(3000, activation='relu', input_dim=86069))
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(15, activation='softmax'))

METRICS = ['accuracy']
# import sgd

model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=METRICS)

model.fit(X_train, y_train,
            epochs=50,
            batch_size=32)

Epoch 1/50
391/391 [==============================] - 10s 20ms/step - loss: 2.3348 - accuracy: 0.3491
Epoch 2/50
391/391 [==============================] - 8s 20ms/step - loss: 2.2088 - accuracy: 0.3786
Epoch 3/50
391/391 [==============================] - 8s 21ms/step - loss: 2.1690 - accuracy: 0.3786
Epoch 4/50
391/391 [==============================] - 8s 20ms/step - loss: 2.1195 - accuracy: 0.3784
Epoch 5/50
391/391 [==============================] - 8s 20ms/step - loss: 2.0672 - accuracy: 0.3785
Epoch 6/50
391/391 [==============================] - 8s 20ms/step - loss: 2.0164 - accuracy: 0.3787
Epoch 7/50
391/391 [==============================] - 8s 20ms/step - loss: 1.9577 - accuracy: 0.3781
Epoch 8/50
391/391 [==============================] - 8s 20ms/step - loss: 1.9283 - accuracy: 0.3767
Epoch 9/50
391/391 [==============================] - 8s 20ms/step - loss: 2.0201 - accuracy: 0.3651
Epoch 10/50
391/391 [==============================] - 8s 20ms/step - loss: 2.3193 - accur

KeyboardInterrupt: 